<center><h1>Amazon Web Scrapping</h1></center>

<h4>1) Importing the libraries</h4>



In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import seaborn as srn
import matplotlib.pyplot as plt

<h4>2) Setting the path of Chrome's driver</h4>

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

<h4>3) Getting our search parameters</h4>

In [3]:
search_term = input('Write the product name')
search_term = search_term.replace(' ','+')  

The user will decide which product the program will scrap data

<h4>4) Iterating between products and pages</h4>

A empty list is created to storage our product data throughout the iterations

In [4]:
list_products = []

The next block of code will navigate between the first page to the last one, and the URL will get the three variables that we defined for our search.

The program will get the source code of the URL for each page (first for loop). The products are inside a tag div with a specific name, the program will iterate through the products (second for loop) and scrap five informations:
1) Product Name
2) Price
3) Rating
4) Review Count
5) Link

(I'm not considering sponsored products)<br>
If it returns AtributeError, the variable will receive a NULL value. The result will be a tuple with our 5 variables and after each product, this tuple will be storaged in the variable 'list_products' which is the empty list created right above.

In [5]:
page = 0
for page in range(1,8):
    url = f'https://www.amazon.com.br/s?k={search_term}&page={page}&__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=32CUIJ6TTV5WT&qid=1646248377&sprefix={search_term}%2Caps%2C275&ref=sr_pg_{page}'    

    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    results = soup.find_all('div', {'data-component-type': 's-search-result'})
    for product in results:
        sponsored_product = product.find('span','a-color-base').text.strip()
        if sponsored_product == "Patrocinados":
            continue
        
        a_tag = product.h2.a
        product_name = a_tag.text.strip()
        link = 'https://www.amazon.com.br' + a_tag.get('href')
        try:
            price_span_tag = product.find('span', 'a-price')
            price = price_span_tag.find('span', 'a-offscreen').text.strip()
        except AttributeError:
            price = ''
                
        try:
            rating_icon = product.find('span','a-icon-alt') .text.strip()
            review_count = product.find('span', {'class': 'a-size-base s-underline-text'}).text
            review_count = str(review_count)
            review_count = review_count.replace('.','')
        except AttributeError:
            rating = ''
            review_count = ''

        result = (product_name, price, rating_icon, review_count, link)
        list_products.append(result)
driver.close()

I previously created a csv file, whose will get the products (list_products) and right after will be transformed in a pandas dataframe

In [6]:
with open('products.csv','w', encoding = 'utf-8') as p:
    writer = csv.writer(p)
    writer.writerow(['PRODUCT DESCRIPTION','PRICE','RATING','REVIEW COUNT','LINK'])
    writer.writerows(list_products)

In [7]:
df = pd.read_csv('products.csv')
df.head()

,PRODUCT DESCRIPTION,PRICE,RATING,REVIEW COUNT,LINK
0,Multilaser - PD588 Pen Drive Twist 16GB USB Le...,"R$ 17,94","4,6 de 5 estrelas",10613.0,https://www.amazon.com.br/Multilaser-Twist1-Me...
1,"Pendrive Datatraveler 100G3 32Gb, Kingston, Pe...","R$ 63,00","4,8 de 5 estrelas",60187.0,https://www.amazon.com.br/Pendrive-DataTravele...
2,"PenDrive SanDisk Cruzer Ultra Flair 32GB, Prata","R$ 80,50","4,6 de 5 estrelas",111966.0,https://www.amazon.com.br/Drive-Cruzer-Ultra-S...
3,MULTILASER Diamond Pd 851 Pen Drive Usb Metáli...,"R$ 54,90","4,5 de 5 estrelas",577.0,https://www.amazon.com.br/MULTILASER-Diamond-D...
4,"Pen Drive Cruzer Blade, SanDisk, 32GB, SDCZ60-...","R$ 39,90","4,8 de 5 estrelas",134756.0,https://www.amazon.com.br/Pen-Drive-32GB-Sandi...


<h4>5) Doing some EDA (Exploratory Data Analysis)</h4>

Checking the type of each column

In [8]:
print(df.dtypes)

PRODUCT DESCRIPTION     object
PRICE                   object
RATING                  object
REVIEW COUNT           float64
LINK                    object
dtype: object


Checking the number of null values in each column

In [9]:
df.isnull().sum()

PRODUCT DESCRIPTION     0
PRICE                  50
RATING                  0
REVIEW COUNT           79
LINK                    0
dtype: int64

Removing the NAn

In [10]:
df = df.dropna()
df = df.reset_index(drop=True)
df.isnull().sum()

PRODUCT DESCRIPTION    0
PRICE                  0
RATING                 0
REVIEW COUNT           0
LINK                   0
dtype: int64

Treating the 'RATING' column to get only the number

In [11]:
df['RATING'] = df['RATING'].str[:3].replace({',':'.'}, regex=True).astype(float)
df.head()

,PRODUCT DESCRIPTION,PRICE,RATING,REVIEW COUNT,LINK
0,Multilaser - PD588 Pen Drive Twist 16GB USB Le...,"R$ 17,94",4.6,10613.0,https://www.amazon.com.br/Multilaser-Twist1-Me...
1,"Pendrive Datatraveler 100G3 32Gb, Kingston, Pe...","R$ 63,00",4.8,60187.0,https://www.amazon.com.br/Pendrive-DataTravele...
2,"PenDrive SanDisk Cruzer Ultra Flair 32GB, Prata","R$ 80,50",4.6,111966.0,https://www.amazon.com.br/Drive-Cruzer-Ultra-S...
3,MULTILASER Diamond Pd 851 Pen Drive Usb Metáli...,"R$ 54,90",4.5,577.0,https://www.amazon.com.br/MULTILASER-Diamond-D...
4,"Pen Drive Cruzer Blade, SanDisk, 32GB, SDCZ60-...","R$ 39,90",4.8,134756.0,https://www.amazon.com.br/Pen-Drive-32GB-Sandi...


The column 'REVIEW COUNT' will get only integer values

In [12]:
#df['REVIEW COUNT'] = df['REVIEW COUNT'].replace({'.':''}, regex=True).astype(int)
df['REVIEW COUNT'] = df['REVIEW COUNT'].astype('int')
#df['REVIEW COUNT'] = df['REVIEW COUNT'].str.replace({'.':','},regex=True)
df.head()

,PRODUCT DESCRIPTION,PRICE,RATING,REVIEW COUNT,LINK
0,Multilaser - PD588 Pen Drive Twist 16GB USB Le...,"R$ 17,94",4.6,10613,https://www.amazon.com.br/Multilaser-Twist1-Me...
1,"Pendrive Datatraveler 100G3 32Gb, Kingston, Pe...","R$ 63,00",4.8,60187,https://www.amazon.com.br/Pendrive-DataTravele...
2,"PenDrive SanDisk Cruzer Ultra Flair 32GB, Prata","R$ 80,50",4.6,111966,https://www.amazon.com.br/Drive-Cruzer-Ultra-S...
3,MULTILASER Diamond Pd 851 Pen Drive Usb Metáli...,"R$ 54,90",4.5,577,https://www.amazon.com.br/MULTILASER-Diamond-D...
4,"Pen Drive Cruzer Blade, SanDisk, 32GB, SDCZ60-...","R$ 39,90",4.8,134756,https://www.amazon.com.br/Pen-Drive-32GB-Sandi...


In [13]:
df.dtypes

PRODUCT DESCRIPTION     object
PRICE                   object
RATING                 float64
REVIEW COUNT             int32
LINK                    object
dtype: object

Removing the 'R$' part from the price

In [14]:
remove_coin = df['PRICE'].str.split('$').str[1]
df['PRICE'] = remove_coin.str.strip()
#price_integer = 


To correct the column 'Price' i had to split the numbers in two parts (integer part and decimal part) and replace ',' to a empty value. Then I had to put these two parts together again with '.' to separate them. It's a not so pretty solution but it worked, and I intend to optimize this part.

<h5>Splitting and Replacing</h5>

In [15]:

df['integer_part'] = df['PRICE'].str.split(',').str[0]
df['integer_part'] = df['integer_part'].str.replace('.','')
df['integer_part']

C:\Users\Alfred\AppData\Local\Temp\ipykernel_46372\1546184746.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['integer_part'] = df['integer_part'].str.replace('.','')


0       17
1       63
2       80
3       54
4       39
      ... 
181    169
182    991
183    171
184     93
185    190
Name: integer_part, Length: 186, dtype: object

In [16]:
df['decimal_part'] = df['PRICE'].str.split(',').str[1]
df['decimal_part']

0      94
1      00
2      50
3      90
4      90
       ..
181    00
182    21
183    65
184    10
185    99
Name: decimal_part, Length: 186, dtype: object

<h5>Joining the parts</h5>

In [17]:
df['PRICE_CORRECT'] = df['integer_part'] + "." + df['decimal_part']
df['PRICE_CORRECT'] = df['PRICE_CORRECT'].astype('string')
df['PRICE_CORRECT'] = df['PRICE_CORRECT'].astype('float')

In [18]:
df['PRICE'] = df['PRICE_CORRECT']
df.drop(['integer_part','decimal_part','PRICE_CORRECT'], axis=1, inplace=True)


Now that the dataframe is polished, let's check how the variables 'Rating' and 'Price' are distributed. It's possible to see that the most part of Power Banks in Amazon have a sort of good quality, due to de high rating. And the Average price is R$100,00 with some outliers. 

In [24]:
import plotly.express as px

In [40]:
fig1 = px.histogram(df['RATING'], color_discrete_sequence=['#520AA9'], title='RATING DISTRIBUTION', width=800, height=350)
fig1.show()

In [41]:
fig2 = px.histogram(df['PRICE'], color_discrete_sequence=['#520AA9'], title='PRICE DISTRIBUTION', width=800, height=350)
fig2.show()

In [42]:
fig3 = px.histogram(df['REVIEW COUNT'], color_discrete_sequence=['#520AA9'], title='REVIEWS DISTRIBUTION', width=800, height=350)
fig3.show()

In [20]:
df.head()

,PRODUCT DESCRIPTION,PRICE,RATING,REVIEW COUNT,LINK
0,Multilaser - PD588 Pen Drive Twist 16GB USB Le...,17.94,4.6,10613,https://www.amazon.com.br/Multilaser-Twist1-Me...
1,"Pendrive Datatraveler 100G3 32Gb, Kingston, Pe...",63.00,4.8,60187,https://www.amazon.com.br/Pendrive-DataTravele...
2,"PenDrive SanDisk Cruzer Ultra Flair 32GB, Prata",80.50,4.6,111966,https://www.amazon.com.br/Drive-Cruzer-Ultra-S...
3,MULTILASER Diamond Pd 851 Pen Drive Usb Metáli...,54.90,4.5,577,https://www.amazon.com.br/MULTILASER-Diamond-D...
4,"Pen Drive Cruzer Blade, SanDisk, 32GB, SDCZ60-...",39.90,4.8,134756,https://www.amazon.com.br/Pen-Drive-32GB-Sandi...


Organizing the data to show the most reviwed and most rated products first

In [43]:
df.sort_values(by=['REVIEW COUNT','RATING'], inplace=True, ascending=False)
df.head(10)

,PRODUCT DESCRIPTION,PRICE,RATING,REVIEW COUNT,LINK
4,"Pen Drive Cruzer Blade, SanDisk, 32GB, SDCZ60-...",39.90,4.8,134756,https://www.amazon.com.br/Pen-Drive-32GB-Sandi...
2,"PenDrive SanDisk Cruzer Ultra Flair 32GB, Prata",80.50,4.6,111966,https://www.amazon.com.br/Drive-Cruzer-Ultra-S...
69,"Pen Drive Ultra, USB 3.0, San Disk, 64GB, SDCZ...",63.79,4.7,101275,https://www.amazon.com.br/Drive-Ultra-Disk-64G...
5,"Pen Drive Ultra Fit SanDisk 3.1, 128GB, SDCZ43...",139.99,4.8,89849,https://www.amazon.com.br/Drive-Ultra-Flash-Sa...
27,SanDisk Flash Drive Ultra USB 3.0 de 512 GB - ...,325.86,4.6,68620,https://www.amazon.com.br/Sandisk-Flash-Memory...
1,"Pendrive Datatraveler 100G3 32Gb, Kingston, Pe...",63.00,4.8,60187,https://www.amazon.com.br/Pendrive-DataTravele...
23,Pen Drive Cruzer Blade - 16GB - PRETO,27.59,4.5,59332,https://www.amazon.com.br/Drive-Sandisk-Cruzer...
35,Samsung BAR Plus 64 GB - 300 MB/s USB 3.1 Flas...,125.41,4.7,39804,https://www.amazon.com.br/Flash-Drive-Samsung-...
64,SanDisk Pen Drive Cruzer USB 2.0 de 256 GB - S...,372.38,4.7,26705,https://www.amazon.com.br/Drive-SanDisk-Cruzer...
30,Samsung MUF-64AB/AM FIT Plus 64 GB - USB 3.1 F...,108.62,4.7,25752,https://www.amazon.com.br/Pendrive-USB-3-1-Sam...
